<a href="https://colab.research.google.com/github/Serena-Wang/Anagrams/blob/master/season_week_kernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow/probability.git

Cloning into 'probability'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 31388 (delta 77), reused 64 (delta 7), pack-reused 31251
Receiving objects: 100% (31388/31388), 51.60 MiB | 26.39 MiB/s, done.
Resolving deltas: 100% (26187/26187), done.


In [0]:
import sys
sys.path.append('/content/probability')

In [3]:
cd /content/probability/tensorflow_probability

/content/probability/tensorflow_probability


In [0]:
#!pip install --upgrade tensorflow

In [0]:
#import probability

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%tensorflow_version 2.x
#import tensorflow as tf
import tensorflow.compat.v2 as tf

from tensorflow_probability.python.internal import assert_util
from tensorflow_probability.python.internal import tensor_util
from tensorflow_probability.python.math.psd_kernels.internal import util
from tensorflow_probability.python.math.psd_kernels import exponentiated_quadratic
from tensorflow_probability.python.math.psd_kernels.positive_semidefinite_kernel import PositiveSemidefiniteKernel


TensorFlow 2.x selected.


In [0]:
__all__ = ['season_week_kernel']

In [0]:
class season_week_kernel(PositiveSemidefiniteKernel):
  """The season_week_kernel kernel.
  Sometimes called the "squared exponential", "Gaussian" or "radial basis
  function", this kernel function has the form
    ```none
    k(x, y) = amplitude**2 * exp(-||x - y||**2 / (2 * length_scale**2))
    ```
  where the double-bars represent vector length (ie, Euclidean, or L2 norm).
  """

  def __init__(self,
               amplitude_k=None,
               length_scale_k=None,
               amplitude_ks=None,
               length_scale_ks=None,
               feature_ndims=1,
               validate_args=False,
               name='season_week_kernel'):
    """Construct an ExponentiatedQuadratic kernel instance.
    Args:
      amplitude: floating point `Tensor` that controls the maximum value
        of the kernel. Must be broadcastable with `length_scale` and inputs to
        `apply` and `matrix` methods. Must be greater than zero. A value of
        `None` is treated like 1.
        Default value: None
      length_scale: floating point `Tensor` that controls how sharp or wide the
        kernel shape is. This provides a characteristic "unit" of length against
        which `||x - y||` can be compared for scale. Must be broadcastable with
        `amplitude` and inputs to `apply` and `matrix` methods. A value of
        `None` is treated like 1.
        Default value: None
      feature_ndims: Python `int` number of rightmost dims to include in the
        squared difference norm in the exponential.
      validate_args: If `True`, parameters are checked for validity despite
        possibly degrading runtime performance
      name: Python `str` name prefixed to Ops created by this class.
    """
    parameters = dict(locals())
    with tf.name_scope(name):
      dtype = util.maybe_get_common_dtype(
          [amplitude_k,amplitude_ks, length_scale_k, length_scale_ks])
      self._amplitude_k = tensor_util.convert_nonref_to_tensor(
          amplitude_k, name='amplitude_k', dtype=dtype)
      self._amplitude_ks = tensor_util.convert_nonref_to_tensor(
          amplitude_ks, name='amplitude_k', dtype=dtype)
      self._length_scale_k = tensor_util.convert_nonref_to_tensor(
          length_scale_k, name='length_scale_ks', dtype=dtype)
      self._length_scale_ks = tensor_util.convert_nonref_to_tensor(
          length_scale_ks, name='length_scale_ks', dtype=dtype)
      
      super(season_week_kernel, self).__init__(
          feature_ndims,
          dtype=dtype,
          name=name,
          validate_args=validate_args,
          parameters=parameters)

  @property
  def amplitude_k(self):
    """Amplitude parameter."""
    return self._amplitude_k

  @property
  def amplitude_ks(self):
      """Amplitude parameter."""
      return self._amplitude_ks

  @property
  def length_scale_k(self):
    """Length scale parameter."""
    return self._length_scale_k
  
  @property
  def length_scale_ks(self):
    """Length scale parameter."""
    return self._length_scale_ks

  ###not sure
  ##This property describes the fully broadcast shape of all kernel parameters
  def _batch_shape(self):
    scalar_shape = tf.TensorShape([])
    return tf.broadcast_static_shape(
        tf.broadcast_static_shape(
          tf.broadcast_static_shape(
            scalar_shape if self.amplitude_k is None else self.amplitude_k.shape,
            scalar_shape if self.length_scale_k is None else self.length_scale_k.shape),
          scalar_shape if self.amplitude_ks is None else self.amplitude_ks.shape),
        scalar_shape if self.length_scale_ks is None else self.length_scale_ks.shape)
        
  ###not sure
  def _batch_shape_tensor(self):
    return tf.broadcast_dynamic_shape(
        tf.broadcast_dynamic_shape(
          tf.broadcast_dynamic_shape(
            [] if self.amplitude_k is None else tf.shape(self.amplitude_k),
            [] if self.length_scale_k is None else tf.shape(self.length_scale_k)),
          [] if self.amplitude_ks is None else tf.shape(self.amplitude_ks),
        [] if self.length_scale_ks is None else tf.shape(self.length_scale_ks)))

  def _apply(self, x1, x2, example_ndims=0):

    if self.amplitude_k is not None:
      amplitude_k = tf.convert_to_tensor(self.amplitude_k)
      amplitude_k = util.pad_shape_with_ones(amplitude_k, example_ndims)

    if self.length_scale_k is not None:
      length_scale_k = tf.convert_to_tensor(self.length_scale_k)
      length_scale_k = util.pad_shape_with_ones(length_scale_k, example_ndims)
      
    if self.amplitude_ks is not None:
      amplitude_ks = tf.convert_to_tensor(self._amplitude_ks)
      amplitude_ks = util.pad_shape_with_ones(amplitude_ks, example_ndims)

    if self.length_scale_ks is not None:
      length_scale_ks = tf.convert_to_tensor(self._length_scale_ks)
      length_scale_ks = util.pad_shape_with_ones(length_scale_ks, example_ndims)
      

    k = tfp.math.psd_kernels.ExponentiatedQuadratic(self.amplitude_k, self.length_scale_k)

    k_s = tfp.math.psd_kernels.ExponentiatedQuadratic(self.amplitude_ks, self.length_scale_ks)

    result = k.apply(x1[:,1:], x2[:,1:], example_ndims=1)
    result_s = k_s.apply(x1[:,1:], x2[:,1:], example_ndims=1)
    #get rows with same seasons
    inds = tf.where(x1[:, 0] == x2[:, 0])

    shape = result.get_shape() 

    print("batch shape = " + str(self._batch_shape()))
    print(inds)
    print(result_s.get_shape())

    delta = tf.SparseTensor(inds, tf.gather_nd(result_s, [inds, :]), shape)
    #print(tf.sparse.to_dense(delta))
    final_result = result + tf.sparse.to_dense(delta)
    return final_result
    #print(final_result)

  def _parameter_control_dependencies(self, is_init):
    if not self.validate_args:
      return []
    assertions = []
    for arg_name, arg in dict(amplitude_k=self.amplitude_k,
                              length_scale_k=self.length_scale_k,amplitude_ks=self.amplitude_ks,
                              length_scale_ks=self.length_scale_ks).items():
      if arg is not None and is_init != tensor_util.is_ref(arg):
        assertions.append(assert_util.assert_positive(
            arg,
            message='{} must be positive.'.format(arg_name)))
    return assertions

Test

In [0]:
import tensorflow_probability as tfp
tfd = tfp.distributions
import numpy as np
import pandas as pd
tfb = tfp.bijectors

In [9]:
np.random.seed(84963)
x = np.concatenate(
  (np.array([0, 0, 0, 1, 1]).reshape((5, 1)), np.linspace(1, 5, 5).reshape((5, 1))),
  axis = 1
)
y = np.concatenate(
  (np.array([0, 1, 1, 1, 2]).reshape((5, 1)), np.linspace(1, 2, 5).reshape((5, 1))),
  axis = 1
)

print(x)
print(y)

[[0. 1.]
 [0. 2.]
 [0. 3.]
 [1. 4.]
 [1. 5.]]
[[0.   1.  ]
 [1.   1.25]
 [1.   1.5 ]
 [1.   1.75]
 [2.   2.  ]]


In [0]:
kernel = season_week_kernel(0.5,1.2,0.7,1.7)

In [42]:
kernel.apply(x, y, example_ndims=1)

batch shape = ()
tf.Tensor(
[[0]
 [3]], shape=(2, 1), dtype=int64)
(5,)


<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.74      , 0.20564438, 0.11445834, 0.24719116, 0.01098423],
      dtype=float32)>

In [43]:
kernel = season_week_kernel([0.5, 0.2],[1.2, 1.7],[0.7, 0.8],[1.7, 2.0])
kernel.apply(x, y, example_ndims=1)

batch shape = (2,)
tf.Tensor(
[[0]
 [3]], shape=(2, 1), dtype=int64)
(2, 5)


InvalidArgumentError: ignored